# Импорт библиотек

In [2]:
import joblib
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

import os

In [29]:
from catboost import CatBoostClassifier
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    ConfusionMatrixDisplay,
)
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler

from sklearn.metrics import (
    roc_auc_score,
    precision_score,
    recall_score,
    f1_score,
    roc_curve,
    confusion_matrix,
)
from sklearn.model_selection import TimeSeriesSplit, cross_val_score

In [5]:
tqdm.pandas()
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
RANDOM_STATE = 1206

# Загрузка данных

## Справочная информация

In [6]:
file_path = "../data/mats/base_scores_dict.joblib"

if os.path.isfile(file_path):
    base_scores_dict = joblib.load(file_path)

In [7]:
base_scores_dict.head()

,item,base_score
0,1T,0.4
1,1S,0.4
2,1Lo,0.5
3,1Eu,0.5
4,1F,0.5


In [8]:
file_path = "../data/mats/nlp_dict.joblib"

if os.path.isfile(file_path):
    nlp_dict = joblib.load(file_path)

In [9]:
nlp_dict.head()

,item,nlp_item,embed_0,embed_1,embed_2,embed_3,embed_4,embed_5,embed_6,embed_7,embed_8,embed_9
0,1T,Single Toeloop,3.158019,-1.617936,0.396813,-1.733884,1.948211,0.765280,-4.662311,2.644467,1.162508,0.099441
1,1S,Single Salchow,3.601343,-2.698863,-0.136379,-0.407949,-1.530264,-2.107504,-2.906400,1.774684,0.182980,-3.096374
2,1Lo,Single Loop,5.122801,-4.158421,-2.667444,0.345399,-0.499262,0.415096,-2.308891,3.640449,-1.090383,2.599226
3,1Eu,Single Euler (only used in jump combinations),0.762735,-3.136916,-0.003426,0.913632,-0.219761,-0.314236,0.426446,-0.346271,2.761745,0.762287
4,1F,Single Flip,4.872679,-4.149351,-1.611438,0.783791,0.614820,-0.363553,-3.880303,1.822208,-0.969000,-0.667879


## Данные для моделирования

In [10]:
file_path = "../data/processed/final_data.joblib"

if os.path.isfile(file_path):
    final_data = joblib.load(file_path)

In [11]:
final_data.shape

(136126, 62)

In [12]:
final_data.columns

Index(['id', 'total_score_id', 'title', 'decrease', 'base_score', 'goe',
       'avg_score', 'unit_id', 'tournament_id', 'base_score_total_scores',
       'components_score', 'total_score', 'elements_score',
       'decreasings_score', 'starting_place', 'place', 'segment_name', 'info',
       'overall_place', 'overall_total_score', 'overall_place_str', 'color',
       'school_id', 'date_start', 'date_end', 'origin_id', 'sequences',
       'cascade', 'title_nlp', 'cascade_nlp', 'multiply',
       'tournament_duration', 'start_month', 'end_month', 'start_day_of_week',
       'end_day_of_week', 'start_is_weekend', 'end_is_weekend', 'start_season',
       'end_season', 'tournament_year', 'units_with_experience', 'falls',
       'components_score_per_element', 'custom_base_score',
       'avg_overall_place_last_year', 'avg_overall_total_score_last_year',
       'avg_components_score_last_year', 'avg_place_last_year',
       'avg_elements_score_last_year', 'avg_decreasings_score_last_year',


In [13]:
file_path = "../data/processed/final_elements_data.joblib"

if os.path.isfile(file_path):
    final_elements_data = joblib.load(file_path)

In [14]:
final_elements_data.shape

(168745, 92)

## Валидационные данные

In [15]:
file_path = "../data/processed/valid_unit_data.joblib"

if os.path.isfile(file_path):
    valid_unit_data = joblib.load(file_path)

In [16]:
file_path = "../data/processed/test_unit_data.joblib"

if os.path.isfile(file_path):
    test_unit_data = joblib.load(file_path)

In [17]:
file_path = "../data/processed/final_test_unit_data.joblib"

if os.path.isfile(file_path):
    final_test_unit_data = joblib.load(file_path)

In [18]:
file_path = "../data/processed/valid_unit_elements_data.joblib"

if os.path.isfile(file_path):
    valid_unit_elements_data = joblib.load(file_path)

In [19]:
file_path = "../data/processed/test_unit_elements_data.joblib"

if os.path.isfile(file_path):
    test_unit_elements_data = joblib.load(file_path)

In [20]:
file_path = "../data/processed/final_test_unit_elements_data.joblib"

if os.path.isfile(file_path):
    final_test_unit_elements_data = joblib.load(file_path)

# Описание идеи

Первая модель должна предсказывать вероятность идеального выполнения выбранного набора элементов или элемента.

В случае работы с наборами элементов (последовательности, каскады) будет предсказываться вероятность идеального исполнения всего набора (дата-сет final_data).

В случае работы поэлементно будет предсказывать вероятность исполнения каждого элемента с учетом его "расположения" в наборе (для этого введены признаки "следующий" и "предыдущий" элемент). Для оценки вероятности идеального исполнения полного набора будет использователь формулу условной вероятности.

Для расчета итоговой вероятности выполнения элементов в каскаде 2A+2F идеально нужно учесть зависимость событий. Вероятность выполнения каждого элемента можно трактовать как условную вероятность, где вероятность выполнения второго элемента зависит от выполнения первого.

Предположим, что вероятность выполнения 2A составляет $P(2A) = 0.7$, а вероятность выполнения 2F после 2A составляет $P(2F \mid 2A) = 0.4$. Таким образом, чтобы вычислить общую вероятность выполнения каскада 2A+2F идеально, нужно умножить вероятности выполнения каждого элемента:

$$
P(2A \text{ и } 2F) = P(2A) \times P(2F \mid 2A)
$$

Подставим значения:

$$
P(2A \text{ и } 2F) = 0.7 \times 0.4 = 0.28
$$

Итак, итоговая вероятность того, что спортсмен выполнит элементы 2A+2F в каскаде идеально, составляет 28%.


На основе предсказания вашей модели можно сделать вывод, что данный элемент фигурного катания в указанном контексте будет выполнен без ошибок с вероятностью 29%. Это означает, что, согласно вашей модели, есть 29% вероятность того, что элемент будет выполнен без ошибок, и 71% вероятность того, что ошибки будут допущены.

### Результаты модели

1. **Целевой признак модели**: выполнение элемента без ошибок (бинарная классификация: 0 - с ошибками, 1 - без ошибок).
2. **Метод `predict`**: возвращает класс, к которому модель относит наблюдение (в данном случае, класс 0 - выполнение элемента с ошибками).
3. **Метод `predict_proba`**: возвращает вероятность того, что наблюдение принадлежит к каждому классу. В вашем случае, вероятность того, что элемент будет выполнен без ошибок, составляет 0.29, или 29%.

### Интерпретация предсказания модели

Предсказание модели основано на обучении на исторических данных, где учитывались элементы фигурного катания, их контекст (в каскаде или одиночный), наличие ошибок в предыдущих и последующих элементах, а также характеристики спортсмена. Вот несколько ключевых моментов:

- **Вероятность 29%**: Это значит, что среди всех подобных случаев (с аналогичными параметрами) только в 29% случаев элемент выполнялся без ошибок.
- **Контекст и характеристики**: Модель учитывает не только сам элемент, но и его контекст (в каскаде или нет) и характеристики спортсмена, что делает предсказание более точным для конкретной ситуации.

### Практическое использование

На практике, можно использовать эту информацию для принятия решений о том, стоит ли включать данный элемент в программу, или, возможно, следует выбрать другой элемент, который имеет более высокую вероятность быть выполненным без ошибок.

### Пример использования результатов

Если модель предсказывает, что вероятность выполнения элемента без ошибок составляет 29%, это говорит о высоком риске допущения ошибок. В зависимости от стратегии тренера и спортсмена, можно:

1. **Избегать высоко рискованных элементов**: Если целью является минимизация ошибок, можно выбрать элементы с более высокой вероятностью выполнения без ошибок.
2. **Тренировка сложных элементов**: Если есть время и ресурсы для улучшения навыков, можно сосредоточиться на тренировке сложных элементов, чтобы повысить вероятность их выполнения без ошибок в будущем.

Таким образом, предсказание вашей модели предоставляет ценную информацию для принятия решений и стратегического планирования в фигурном катании.

# Моделирование

## Поэлементный подход - дата-сет `final_elements_data`

In [21]:
final_elements_data.columns

Index(['id', 'total_score_id', 'title', 'decrease', 'base_score', 'goe',
       'avg_score', 'unit_id', 'tournament_id', 'base_score_total_scores',
       'components_score', 'total_score', 'elements_score',
       'decreasings_score', 'starting_place', 'place', 'segment_name', 'info',
       'overall_place', 'overall_total_score', 'overall_place_str', 'color',
       'school_id', 'date_start', 'date_end', 'origin_id', 'sequences',
       'cascade', 'title_nlp', 'cascade_nlp', 'multiply',
       'tournament_duration', 'start_month', 'end_month', 'start_day_of_week',
       'end_day_of_week', 'start_is_weekend', 'end_is_weekend', 'start_season',
       'end_season', 'tournament_year', 'units_with_experience', 'falls',
       'components_score_per_element', 'custom_base_score',
       'avg_overall_place_last_year', 'avg_overall_total_score_last_year',
       'avg_components_score_last_year', 'avg_place_last_year',
       'avg_elements_score_last_year', 'avg_decreasings_score_last_year',


Среди всех признаков дата-сета некоторые могут являться причиной утечки целевого признака, а именно: 'decrease', 'base_score', 'goe', 'avg_score', 'attr_element'

Кроме того, часть признаков не может быть определена для тестовых данных: 'decrease', 'goe', 'avg_score'

Целевым признаком для разрабатываемой модели бинарной классификации является: 'target_clear_element'

In [22]:
# Комменитрование признаков, не используемых в моделировании связано
# с удобством проведения большого количества экспериментов
data_e1 = final_elements_data[
    [
        # "id",
        # "total_score_id",
        # "title",
        # "decrease",
        # "base_score",
        # "goe",
        # "avg_score",
        # "unit_id",
        # "tournament_id",
        # "base_score_total_scores",
        # "components_score",
        # "total_score",
        # "elements_score",
        # "decreasings_score",
        # "starting_place",
        # "place",
        "segment_name",
        # "info",
        # "overall_place",
        # "overall_total_score",
        # "overall_place_str",
        "color",
        "school_id",
        # "date_start",
        # "date_end",
        "origin_id",
        # "sequences",
        # "cascade",
        # "title_nlp",
        # "cascade_nlp",
        "multiply",
        "tournament_duration",
        "start_month",
        "end_month",
        "start_day_of_week",
        "end_day_of_week",
        "start_is_weekend",
        "end_is_weekend",
        "start_season",
        "end_season",
        "tournament_year",
        # "units_with_experience",
        "falls",
        # "components_score_per_element",
        # "custom_base_score",
        "avg_overall_place_last_year",
        "avg_overall_total_score_last_year",
        "avg_components_score_last_year",
        "avg_place_last_year",
        "avg_elements_score_last_year",
        "avg_decreasings_score_last_year",
        "avg_total_score_last_year",
        "avg_falls_last_year",
        "target_clear_element",
        "difficulty",
        # "perfect_element",
        # "q_element",
        # "e_element",
        # "l_element",
        # "ll_element",
        # "h_element",
        # "v_element",
        "element",
        # "attr_element",
        "prev_element",
        "attr_prev_element",
        "next_element",
        "attr_next_element",
        "single_element",
        "clear_prev_element",
        "clear_next_element",
        # "perfect_attr_element",
        # "q_attr_element",
        # "e_attr_element",
        # "l_attr_element",
        # "ll_attr_element",
        # "h_attr_element",
        # "v_attr_element",
        # "perfect_attr_prev_element",
        # "q_attr_prev_element",
        # "e_attr_prev_element",
        # "l_attr_prev_element",
        # "ll_attr_prev_element",
        # "h_attr_prev_element",
        # "v_attr_prev_element",
        # "perfect_attr_next_element",
        # "q_attr_next_element",
        # "e_attr_next_element",
        # "l_attr_next_element",
        # "ll_attr_next_element",
        # "h_attr_next_element",
        # "v_attr_next_element",
    ]
].copy()
data_e1.shape


(168745, 34)

In [23]:
data_e1.duplicated(keep="first").sum()

2765

Удаление дубликатов в данных временных рядов – это деликатный вопрос и зависит от контекста данных и цели анализа. Удалять дубликаты целесообразно, когда они появились в результате очевидных ошибок или они не содержат новой информации и не искажают временной ряд. В некоторых временных рядах дубликаты могут отражать реальные повторяющиеся события. Например, в данных о транзакциях может быть несколько одинаковых транзакций в одном и том же временном интервале. Если дубликаты представляют значимую информацию о последовательности событий, их удаление может привести к потере информации. Например, в данных о трафике интернета дублированные записи могут указывать на повышенную активность в определенные моменты времени.

Принято решение не удалять дубликаты, но во всех экспериментах оценивать их количество.

In [24]:
data_e1["target_clear_element"].value_counts()

target_clear_element
1    138436
0     30309
Name: count, dtype: int64

Имеем дисбаланс классов.

Для учета качества предсказания класса 0 выбрана метрика **F1_Weighted** и оценка Classification Report.

### Разделение данных

In [25]:
X = data_e1.drop("target_clear_element", axis=1)
y = data_e1["target_clear_element"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, shuffle=False, random_state=1206
)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(126558, 33)
(126558,)
(42187, 33)
(42187,)


### Моделирование

Определим категориальные и количественные признаки. Для линейной модели применим OHE, для иных - OrdinalEncoder. Во всех случаях, кроме CatBoost используем также StandardScaler для количественных признаков.

In [26]:
category_columns = [
    # "id",
    # "total_score_id",
    # "title",
    # "decrease",
    # "base_score",
    # "goe",
    # "avg_score",
    # "unit_id",
    # "tournament_id",
    # "base_score_total_scores",
    # "components_score",
    # "total_score",
    # "elements_score",
    # "decreasings_score",
    # "starting_place",
    # "place",
    "segment_name",
    # "info",
    # "overall_place",
    # "overall_total_score",
    # "overall_place_str",
    "color",
    "school_id",
    # "date_start",
    # "date_end",
    "origin_id",
    # "sequences",
    # "cascade",
    # "title_nlp",
    # "cascade_nlp",
    # "multiply",
    # "tournament_duration",
    "start_month",
    "end_month",
    "start_day_of_week",
    "end_day_of_week",
    "start_is_weekend",
    "end_is_weekend",
    "start_season",
    "end_season",
    "tournament_year",
    # "units_with_experience",
    "falls",
    # "components_score_per_element",
    # # "custom_base_score",
    # "avg_overall_place_last_year",
    # "avg_overall_total_score_last_year",
    # "avg_components_score_last_year",
    # "avg_place_last_year",
    # "avg_elements_score_last_year",
    # "avg_decreasings_score_last_year",
    # "avg_total_score_last_year",
    # "avg_falls_last_year",
    # "target_clear_element",
    # "difficulty",
    # "perfect_element",
    # "q_element",
    # "e_element",
    # "l_element",
    # "ll_element",
    # "h_element",
    # "v_element",
    "element",
    # "attr_element",
    "prev_element",
    "attr_prev_element",
    "next_element",
    "attr_next_element",
    "single_element",
    "clear_prev_element",
    "clear_next_element",
    # "perfect_attr_element",
    # "q_attr_element",
    # "e_attr_element",
    # "l_attr_element",
    # "ll_attr_element",
    # "h_attr_element",
    # "v_attr_element",
    # "perfect_attr_prev_element",
    # "q_attr_prev_element",
    # "e_attr_prev_element",
    # "l_attr_prev_element",
    # "ll_attr_prev_element",
    # "h_attr_prev_element",
    # "v_attr_prev_element",
    # "perfect_attr_next_element",
    # "q_attr_next_element",
    # "e_attr_next_element",
    # "l_attr_next_element",
    # "ll_attr_next_element",
    # "h_attr_next_element",
    # "v_attr_next_element",
]

ohe_columns = [x for x in category_columns if x in X.columns]
scale_columns = [x for x in X.columns if x not in category_columns]

#### Линейная регрессия

In [28]:
preprocessor_lr = ColumnTransformer(
    transformers=[
        ("ohe", OneHotEncoder(drop="first", handle_unknown="ignore"), ohe_columns),
        ("scaler", StandardScaler(), scale_columns),
    ]
)

model_lr = make_pipeline(preprocessor_lr, LogisticRegression(max_iter=1000))


In [30]:
tscv = TimeSeriesSplit(n_splits=3)

In [32]:
cv_result = cross_val_score(
    model_lr, X_train, y_train, cv=tscv, scoring="f1_weighted", verbose=500
)


[CV] START .....................................................................


c:\Users\filat\Documents\projects\my_champion_app\.venv\Lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [2, 4, 5, 14, 15] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


[CV] END ................................ score: (test=0.829) total time=   0.8s
[Parallel(n_jobs=1)]: Done   1 tasks      | elapsed:    0.8s
[CV] START .....................................................................


c:\Users\filat\Documents\projects\my_champion_app\.venv\Lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [2, 4, 5, 14, 18] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


[CV] END ................................ score: (test=0.816) total time=   1.6s
[Parallel(n_jobs=1)]: Done   2 tasks      | elapsed:    2.5s
[CV] START .....................................................................
[CV] END ................................ score: (test=0.824) total time=   2.4s
[Parallel(n_jobs=1)]: Done   3 tasks      | elapsed:    5.0s
[Parallel(n_jobs=1)]: Done   3 tasks      | elapsed:    5.0s


c:\Users\filat\Documents\projects\my_champion_app\.venv\Lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [2, 12, 15] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [34]:
print("Среднее значение метрики: " f"{cv_result.mean():.2f}")
print("Стандартное отклонение предсказаний: " f"{cv_result.std():.2f}")


Среднее значение метрики: 0.82
Стандартное отклонение предсказаний: 0.01


#### KNeighborsClassifier

In [42]:
preprocessor_knn = ColumnTransformer(
    transformers=[
        (
            "ohe",
            OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1),
            ohe_columns,
        ),
        ("scaler", StandardScaler(), scale_columns),
    ]
)

model_knn = make_pipeline(preprocessor_knn, KNeighborsClassifier())


In [43]:
cv_result = cross_val_score(
    model_knn, X_train, y_train, cv=tscv, scoring="f1_weighted", verbose=500
)


[CV] START .....................................................................
[CV] END ................................ score: (test=0.797) total time=   1.6s
[Parallel(n_jobs=1)]: Done   1 tasks      | elapsed:    1.6s
[CV] START .....................................................................
[CV] END ................................ score: (test=0.798) total time=   2.5s
[Parallel(n_jobs=1)]: Done   2 tasks      | elapsed:    4.1s
[CV] START .....................................................................
[CV] END ................................ score: (test=0.799) total time=   3.6s
[Parallel(n_jobs=1)]: Done   3 tasks      | elapsed:    7.9s
[Parallel(n_jobs=1)]: Done   3 tasks      | elapsed:    7.9s


In [44]:
print("Среднее значение метрики: " f"{cv_result.mean():.2f}")
print("Стандартное отклонение предсказаний: " f"{cv_result.std():.2f}")


Среднее значение метрики: 0.80
Стандартное отклонение предсказаний: 0.00


#### RandomForestClassifier

In [45]:
preprocessor_rfc = ColumnTransformer(
    transformers=[
        (
            "ohe",
            OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1),
            ohe_columns,
        ),
        ("scaler", StandardScaler(), scale_columns),
    ]
)

model_rfc = make_pipeline(preprocessor_rfc, RandomForestClassifier())


In [46]:
cv_result = cross_val_score(
    model_rfc, X_train, y_train, cv=tscv, scoring="f1_weighted", verbose=500
)


[CV] START .....................................................................
[CV] END ................................ score: (test=0.837) total time=   4.7s
[Parallel(n_jobs=1)]: Done   1 tasks      | elapsed:    4.7s
[CV] START .....................................................................
[CV] END ................................ score: (test=0.841) total time=  10.5s
[Parallel(n_jobs=1)]: Done   2 tasks      | elapsed:   15.3s
[CV] START .....................................................................
[CV] END ................................ score: (test=0.836) total time=  17.1s
[Parallel(n_jobs=1)]: Done   3 tasks      | elapsed:   32.5s
[Parallel(n_jobs=1)]: Done   3 tasks      | elapsed:   32.5s


In [47]:
print("Среднее значение метрики: " f"{cv_result.mean():.2f}")
print("Стандартное отклонение предсказаний: " f"{cv_result.std():.2f}")


Среднее значение метрики: 0.84
Стандартное отклонение предсказаний: 0.00


#### CatBoost

In [52]:
model_e1_ctr = CatBoostClassifier(
    verbose=50,
    n_estimators=150,
    cat_features=category_columns,
    random_state=RANDOM_STATE,
)

In [53]:
cv_result = cross_val_score(
    model_e1_ctr, X_train, y_train, cv=tscv, scoring="f1_weighted", verbose=500
)


[CV] START .....................................................................
Learning rate set to 0.256487
0:	learn: 0.5214450	total: 70.8ms	remaining: 10.5s
50:	learn: 0.2753524	total: 3.21s	remaining: 6.23s
100:	learn: 0.2561789	total: 6.3s	remaining: 3.06s
149:	learn: 0.2425230	total: 9.28s	remaining: 0us
[CV] END ................................ score: (test=0.842) total time=   9.9s
[Parallel(n_jobs=1)]: Done   1 tasks      | elapsed:    9.9s
[CV] START .....................................................................
Learning rate set to 0.344825
0:	learn: 0.4756102	total: 76.4ms	remaining: 11.4s
50:	learn: 0.2774270	total: 3.73s	remaining: 7.24s
100:	learn: 0.2624868	total: 7.31s	remaining: 3.54s
149:	learn: 0.2503013	total: 10.9s	remaining: 0us
[CV] END ................................ score: (test=0.842) total time=  11.8s
[Parallel(n_jobs=1)]: Done   2 tasks      | elapsed:   21.8s
[CV] START .....................................................................
Learni

In [54]:
print("Среднее значение метрики: " f"{cv_result.mean():.2f}")
print("Стандартное отклонение предсказаний: " f"{cv_result.std():.2f}")


Среднее значение метрики: 0.84
Стандартное отклонение предсказаний: 0.00


#### Тестрирование

Для тестирования выберем CatBoost

In [55]:
model_e1_ctr.fit(X_train, y_train)


Learning rate set to 0.463592
0:	learn: 0.4463139	total: 91.8ms	remaining: 13.7s
50:	learn: 0.2836492	total: 4.98s	remaining: 9.68s
100:	learn: 0.2719430	total: 9.97s	remaining: 4.84s
149:	learn: 0.2623766	total: 14.8s	remaining: 0us


In [56]:
y_pred = model_e1_ctr.predict(X_test)
y_proba = model_e1_ctr.predict_proba(X_test)


In [57]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.71      0.50      0.58      8847
           1       0.88      0.94      0.91     33340

    accuracy                           0.85     42187
   macro avg       0.79      0.72      0.75     42187
weighted avg       0.84      0.85      0.84     42187



#### Важность признаков

In [58]:
importances = model_e1_ctr.get_feature_importance()

feature_importance_df = pd.DataFrame(
    {"feature": X_train.columns, "importance": importances}
).sort_values(by="importance", ascending=False)

feature_importance_df

,feature,importance
25,element,36.142924
24,difficulty,17.471268
17,avg_overall_total_score_last_year,8.507504
0,segment_name,4.352867
20,avg_elements_score_last_year,3.963735
2,school_id,3.236039
15,falls,2.902475
3,origin_id,2.306661
26,prev_element,2.017422
18,avg_components_score_last_year,1.889652


Выберем признаки с важностью более 1 и переобучим модель.

In [62]:
selected_features = feature_importance_df[feature_importance_df["importance"] > 1][
    "feature"
].to_list()
selected_features

['element',
 'difficulty',
 'avg_overall_total_score_last_year',
 'segment_name',
 'avg_elements_score_last_year',
 'school_id',
 'falls',
 'origin_id',
 'prev_element',
 'avg_components_score_last_year',
 'avg_overall_place_last_year',
 'next_element',
 'avg_falls_last_year',
 'avg_place_last_year',
 'avg_total_score_last_year',
 'tournament_duration']

In [63]:
cat_features = [x for x in category_columns if x in selected_features]

In [65]:
model_e1_ctr_1 = CatBoostClassifier(
    verbose=50,
    n_estimators=150,
    cat_features=cat_features,
    random_state=RANDOM_STATE,
)

model_e1_ctr_1.fit(X_train[selected_features], y_train)

Learning rate set to 0.463592
0:	learn: 0.5058352	total: 61ms	remaining: 9.08s
50:	learn: 0.2931011	total: 2.98s	remaining: 5.79s
100:	learn: 0.2834297	total: 5.82s	remaining: 2.82s
149:	learn: 0.2780111	total: 8.62s	remaining: 0us


In [66]:
cv_result = cross_val_score(
    model_e1_ctr_1,
    X_train[selected_features],
    y_train,
    cv=tscv,
    scoring="f1_weighted",
    verbose=500,
)


[CV] START .....................................................................
Learning rate set to 0.256487
0:	learn: 0.5637632	total: 45.4ms	remaining: 6.77s
50:	learn: 0.2868223	total: 2.28s	remaining: 4.43s
100:	learn: 0.2761221	total: 4.46s	remaining: 2.16s
149:	learn: 0.2671675	total: 6.64s	remaining: 0us
[CV] END ................................ score: (test=0.845) total time=   6.8s
[Parallel(n_jobs=1)]: Done   1 tasks      | elapsed:    6.8s
[CV] START .....................................................................
Learning rate set to 0.344825
0:	learn: 0.5362913	total: 95.1ms	remaining: 14.2s
50:	learn: 0.2873367	total: 2.52s	remaining: 4.89s
100:	learn: 0.2763215	total: 4.91s	remaining: 2.38s
149:	learn: 0.2693828	total: 7.41s	remaining: 0us
[CV] END ................................ score: (test=0.848) total time=   7.8s
[Parallel(n_jobs=1)]: Done   2 tasks      | elapsed:   14.7s
[CV] START .....................................................................
Learn

In [67]:
print("Среднее значение метрики: " f"{cv_result.mean():.2f}")
print("Стандартное отклонение предсказаний: " f"{cv_result.std():.2f}")


Среднее значение метрики: 0.84
Стандартное отклонение предсказаний: 0.00


In [68]:
y_pred_e1_ctr_1 = model_e1_ctr_1.predict(X_test[selected_features])
y_proba_e1_ctr_1 = model_e1_ctr_1.predict_proba(X_test[selected_features])

In [69]:
f1_score_weighted_e1_ctr_1 = f1_score(y_test, y_pred_e1_ctr_1, average="weighted")
f1_score_weighted_e1_ctr_1

0.8404989503111845

In [70]:
print(classification_report(y_test, y_pred_e1_ctr_1))

              precision    recall  f1-score   support

           0       0.70      0.50      0.58      8847
           1       0.88      0.94      0.91     33340

    accuracy                           0.85     42187
   macro avg       0.79      0.72      0.75     42187
weighted avg       0.84      0.85      0.84     42187



In [72]:
# joblib.dump(model_e1_ctr, "../../models/model_one.joblib")

Можно поиграть с порогом, для более полного определения класса 0. Пока оставим так.

Попробуем тоже самое, но с NLP-эмбеддингами

## Поэлементный подход с NLP - дата-сет `final_elements_data`

In [73]:
data_e2 = data_e1.copy()
data_e2.shape

(168745, 34)

In [74]:
data_e2 = pd.merge(data_e2, nlp_dict, left_on="element", right_on="item", how="left")
data_e2.shape

(168745, 46)

In [75]:
data_e2 = pd.merge(
    data_e2,
    nlp_dict,
    left_on="next_element",
    right_on="item",
    how="left",
    suffixes=("", "_next"),
)
data_e2.shape

(168745, 58)

In [76]:
data_e2 = pd.merge(
    data_e2,
    nlp_dict,
    left_on="prev_element",
    right_on="item",
    how="left",
    suffixes=("", "_prev"),
)
data_e2.shape

(168745, 70)

In [77]:
data_e2 = data_e2.drop(
    [
        "element",
        "prev_element",
        "next_element",
        "item",
        "nlp_item",
        "item_next",
        "nlp_item_next",
        "item_prev",
        "nlp_item_prev",
    ],
    axis=1,
)

In [78]:
data_e2.duplicated(keep="first").sum()

2765

In [79]:
data_e2["target_clear_element"].value_counts()

target_clear_element
1    138436
0     30309
Name: count, dtype: int64

### Разделение данных

In [87]:
X = data_e2.drop("target_clear_element", axis=1)
y = data_e2["target_clear_element"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, shuffle=False, random_state=1206
)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(126558, 60)
(126558,)
(42187, 60)
(42187,)


### Моделирование

In [88]:
category_columns = [
    # "id",
    # "total_score_id",
    # "title",
    # "decrease",
    # "base_score",
    # "goe",
    # "avg_score",
    # "unit_id",
    # "tournament_id",
    # "base_score_total_scores",
    # "components_score",
    # "total_score",
    # "elements_score",
    # "decreasings_score",
    # "starting_place",
    # "place",
    "segment_name",
    # "info",
    # "overall_place",
    # "overall_total_score",
    # "overall_place_str",
    "color",
    "school_id",
    # "date_start",
    # "date_end",
    "origin_id",
    # "sequences",
    # "cascade",
    # "title_nlp",
    # "cascade_nlp",
    "multiply",
    # "tournament_duration",
    "start_month",
    "end_month",
    "start_day_of_week",
    "end_day_of_week",
    "start_is_weekend",
    "end_is_weekend",
    "start_season",
    "end_season",
    "tournament_year",
    # "units_with_experience",
    "falls",
    # "components_score_per_element",
    # "custom_base_score",
    # "avg_overall_place_last_year",
    # "avg_overall_total_score_last_year",
    # "avg_components_score_last_year",
    # "avg_place_last_year",
    # "avg_elements_score_last_year",
    # "avg_decreasings_score_last_year",
    # "avg_total_score_last_year",
    # "avg_falls_last_year",
    # "element",
    # "attr_element",
    # "prev_element",
    "attr_prev_element",
    # "next_element",
    "attr_next_element",
    "single_element",
    # "target_clear_element",
    "clear_prev_element",
    "clear_next_element",
    # "difficulty",
    # "perfect_attr_element",
    # "q_attr_element",
    # "e_attr_element",
    # "l_attr_element",
    # "ll_attr_element",
    # "h_attr_element",
    # "v_attr_element",
    # "perfect_attr_prev_element",
    # "q_attr_prev_element",
    # "e_attr_prev_element",
    # "l_attr_prev_element",
    # "ll_attr_prev_element",
    # "h_attr_prev_element",
    # "v_attr_prev_element",
    # "perfect_attr_next_element",
    # "q_attr_next_element",
    # "e_attr_next_element",
    # "l_attr_next_element",
    # "ll_attr_next_element",
    # "h_attr_next_element",
    # "v_attr_next_element",
    # "embed_0",
    # "embed_1",
    # "embed_2",
    # "embed_3",
    # "embed_4",
    # "embed_5",
    # "embed_6",
    # "embed_7",
    # "embed_8",
    # "embed_9",
    # "embed_0_next",
    # "embed_1_next",
    # "embed_2_next",
    # "embed_3_next",
    # "embed_4_next",
    # "embed_5_next",
    # "embed_6_next",
    # "embed_7_next",
    # "embed_8_next",
    # "embed_9_next",
    # "embed_0_prev",
    # "embed_1_prev",
    # "embed_2_prev",
    # "embed_3_prev",
    # "embed_4_prev",
    # "embed_5_prev",
    # "embed_6_prev",
    # "embed_7_prev",
    # "embed_8_prev",
    # "embed_9_prev",
]

ohe_columns = [x for x in category_columns if x in X.columns]
scale_columns = [x for x in X.columns if x not in category_columns]

In [89]:
model_e2_ctc = CatBoostClassifier(
    verbose=50,
    n_estimators=150,
    cat_features=category_columns,
    random_state=RANDOM_STATE,
)

In [90]:
cv_result = cross_val_score(
    model_e2_ctc, X_train, y_train, cv=tscv, scoring="f1_weighted", verbose=500
)


[CV] START .....................................................................
Learning rate set to 0.256487
0:	learn: 0.5053774	total: 67.5ms	remaining: 10.1s
50:	learn: 0.2743041	total: 3.58s	remaining: 6.95s
100:	learn: 0.2472491	total: 7.02s	remaining: 3.4s
149:	learn: 0.2310696	total: 10.4s	remaining: 0us
[CV] END ................................ score: (test=0.851) total time=  11.0s
[Parallel(n_jobs=1)]: Done   1 tasks      | elapsed:   11.0s
[CV] START .....................................................................
Learning rate set to 0.344825
0:	learn: 0.4716112	total: 72.6ms	remaining: 10.8s
50:	learn: 0.2726181	total: 3.97s	remaining: 7.71s
100:	learn: 0.2563880	total: 7.94s	remaining: 3.85s
149:	learn: 0.2444515	total: 11.7s	remaining: 0us
[CV] END ................................ score: (test=0.844) total time=  12.5s
[Parallel(n_jobs=1)]: Done   2 tasks      | elapsed:   23.6s
[CV] START .....................................................................
Learni

In [91]:
print("Среднее значение метрики: " f"{cv_result.mean():.2f}")
print("Стандартное отклонение предсказаний: " f"{cv_result.std():.2f}")


Среднее значение метрики: 0.85
Стандартное отклонение предсказаний: 0.00


Заметно улучшение метрики на кросс-валидации.

#### Тестрирование

Для тестирования выберем CatBoost

In [92]:
model_e2_ctc.fit(X_train, y_train)


Learning rate set to 0.463592
0:	learn: 0.4168395	total: 95.4ms	remaining: 14.2s
50:	learn: 0.2823758	total: 5.24s	remaining: 10.2s
100:	learn: 0.2694687	total: 10.3s	remaining: 4.99s
149:	learn: 0.2598861	total: 15.1s	remaining: 0us


In [93]:
y_pred = model_e2_ctc.predict(X_test)
y_proba = model_e2_ctc.predict_proba(X_test)


In [94]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.71      0.49      0.58      8847
           1       0.88      0.95      0.91     33340

    accuracy                           0.85     42187
   macro avg       0.79      0.72      0.75     42187
weighted avg       0.84      0.85      0.84     42187



In [95]:
f1_score_weighted_e2_ctc = f1_score(y_test, y_pred, average="weighted")
f1_score_weighted_e2_ctc

0.8408500589029907

Заметного улучшения на тестовых данных не замечено.

Попробуем "наборный" подход. Используем данные каскадов, комбинаций без разделения на отдельные элементы.

## Наборный подход - дата-сет `final_data`

In [96]:
data_e3 = final_data[
    [
        # "id",
        # "total_score_id",
        # "title",
        # "decrease",
        # "base_score",
        # "goe",
        # "avg_score",
        # "unit_id",
        # "tournament_id",
        # "base_score_total_scores",
        # "components_score",
        # "total_score",
        # "elements_score",
        # "decreasings_score",
        # "starting_place",
        # "place",
        "segment_name",
        # "info",
        # "overall_place",
        # "overall_total_score",
        # "overall_place_str",
        "color",
        "school_id",
        # "date_start",
        # "date_end",
        "origin_id",
        # "sequences",
        "cascade",
        # "title_nlp",
        # "cascade_nlp",
        "multiply",
        "tournament_duration",
        "start_month",
        "end_month",
        "start_day_of_week",
        "end_day_of_week",
        "start_is_weekend",
        "end_is_weekend",
        "start_season",
        "end_season",
        "tournament_year",
        # "units_with_experience",
        "falls",
        # "components_score_per_element",
        # "custom_base_score",
        "avg_overall_place_last_year",
        "avg_overall_total_score_last_year",
        "avg_components_score_last_year",
        "avg_place_last_year",
        "avg_elements_score_last_year",
        "avg_decreasings_score_last_year",
        "avg_total_score_last_year",
        "avg_falls_last_year",
        "target_clear_element",
        "difficulty",
        # "perfect_element",
        # "q_element",
        # "e_element",
        # "l_element",
        # "ll_element",
        # "h_element",
        # "v_element",
    ]
].copy()
data_e3.shape

(136126, 27)

In [97]:
data_e3.duplicated(keep="first").sum()

2431

In [98]:
data_e3["target_clear_element"].value_counts()

target_clear_element
1    108199
0     27927
Name: count, dtype: int64

### Разделение данных

In [99]:
X = data_e3.drop("target_clear_element", axis=1)
y = data_e3["target_clear_element"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, shuffle=False, random_state=1206
)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(102094, 26)
(102094,)
(34032, 26)
(34032,)


### Моделирование

In [100]:
category_columns = [
    # "id",
    # "total_score_id",
    # "title",
    # "decrease",
    # "base_score",
    # "goe",
    # "avg_score",
    # "unit_id",
    # "tournament_id",
    # "base_score_total_scores",
    # "components_score",
    # "total_score",
    # "elements_score",
    # "decreasings_score",
    # "starting_place",
    # "place",
    "segment_name",
    # "info",
    # "overall_place",
    # "overall_total_score",
    # "overall_place_str",
    "color",
    "school_id",
    # "date_start",
    # "date_end",
    "origin_id",
    # "sequences",
    "cascade",
    # "title_nlp",
    # "cascade_nlp",
    # "multiply",
    # "tournament_duration",
    "start_month",
    "end_month",
    "start_day_of_week",
    "end_day_of_week",
    "start_is_weekend",
    "end_is_weekend",
    "start_season",
    "end_season",
    "tournament_year",
    # "units_with_experience",
    "falls",
    # "components_score_per_element",
    # "custom_base_score",
    # "avg_overall_place_last_year",
    # "avg_overall_total_score_last_year",
    # "avg_components_score_last_year",
    # "avg_place_last_year",
    # "avg_elements_score_last_year",
    # "avg_decreasings_score_last_year",
    # "avg_total_score_last_year",
    # "avg_falls_last_year",
    # "target_clear_element",
    # "difficulty",
    # "perfect_element",
    # "q_element",
    # "e_element",
    # "l_element",
    # "ll_element",
    # "h_element",
    # "v_element",
]

ohe_columns = [x for x in category_columns if x in X.columns]
scale_columns = [x for x in X.columns if x not in category_columns]

In [101]:
model_e3_ctс = CatBoostClassifier(
    verbose=50,
    n_estimators=150,
    cat_features=category_columns,
    random_state=RANDOM_STATE,
)

In [102]:
cv_result = cross_val_score(
    model_e3_ctс, X_train, y_train, cv=tscv, scoring="f1_weighted", verbose=500
)


[CV] START .....................................................................
Learning rate set to 0.234009
0:	learn: 0.5241410	total: 67ms	remaining: 9.99s
50:	learn: 0.2938095	total: 3.46s	remaining: 6.71s
100:	learn: 0.2709476	total: 6.78s	remaining: 3.29s
149:	learn: 0.2566208	total: 10.1s	remaining: 0us
[CV] END ................................ score: (test=0.825) total time=  10.4s
[Parallel(n_jobs=1)]: Done   1 tasks      | elapsed:   10.4s
[CV] START .....................................................................
Learning rate set to 0.314605
0:	learn: 0.4918866	total: 71.6ms	remaining: 10.7s
50:	learn: 0.2948923	total: 3.75s	remaining: 7.28s
100:	learn: 0.2763060	total: 7.38s	remaining: 3.58s
149:	learn: 0.2627089	total: 10.9s	remaining: 0us
[CV] END ................................ score: (test=0.824) total time=  11.3s
[Parallel(n_jobs=1)]: Done   2 tasks      | elapsed:   21.9s
[CV] START .....................................................................
Learnin

In [104]:
print("Среднее значение метрики: " f"{cv_result.mean():.2f}")
print("Стандартное отклонение предсказаний: " f"{cv_result.std():.2f}")


Среднее значение метрики: 0.83
Стандартное отклонение предсказаний: 0.00


Качество на кросс-валидации хуже.

#### Тестрирование

In [106]:
model_e3_ctс.fit(X_train, y_train)

Learning rate set to 0.422962
0:	learn: 0.4654640	total: 96.9ms	remaining: 14.4s
50:	learn: 0.2999686	total: 4.8s	remaining: 9.32s
100:	learn: 0.2867286	total: 9.23s	remaining: 4.48s
149:	learn: 0.2766964	total: 13.7s	remaining: 0us


In [107]:
y_pred = model_e3_ctс.predict(X_test)
y_proba = model_e3_ctс.predict_proba(X_test)


In [108]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.70      0.54      0.61      8168
           1       0.87      0.93      0.90     25864

    accuracy                           0.84     34032
   macro avg       0.78      0.74      0.75     34032
weighted avg       0.83      0.84      0.83     34032



In [109]:
f1_score_weighted_e3_ctc = f1_score(y_test, y_pred, average="weighted")
f1_score_weighted_e3_ctc


0.8275894248999055

# Вывод по разработке "Модели № 1"